In [1]:
import pandas as pd
import numpy as np
import pickle
import treecorr
from tqdm import tqdm_notebook as tqdm

In [2]:
df_all=[None]*4

In [3]:
for i in range(4):
    df_all[i]=pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(i+2))

In [4]:
df_riz = pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(3)) 

In [5]:
keys = df_riz.columns[40:]
# keys=['ebv_planck13']

In [6]:
true_df=pd.read_pickle("/home/cz136/project/balrog/data/true_df.pkl")
mcal_list = [df_all]
df_riz_list= [df_riz]

In [7]:
def add_tilename(df_list,true):
    for i in range(4):
        df_list[i]=pd.concat([df_list[i],true_df[['meas_tilename']]],axis=1,join='inner')

In [8]:
for df_list in mcal_list:
    add_tilename(df_list,true_df)

In [9]:
df_riz=pd.concat([df_riz,true_df[['meas_tilename']]],axis=1,join='inner')

In [10]:
data_len=len(df_riz)
wpos1=np.ones(data_len)
w_=np.zeros(data_len)

In [11]:
def get_ke(catalog,key):
    
    
    delta_g1=catalog['e_1'].to_numpy()
    delta_g2=catalog['e_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()

    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process(kcat,gcat)
    xi=kg.xi*kg.npairs
    return (np.array([xi,kg.npairs]))

In [12]:
def get_ke_cross(catalog1,catalog2,key):
    


    k=catalog1[key].to_numpy()
    
    delta_g1=catalog2['e_1'].to_numpy()
    delta_g2=catalog2['e_2'].to_numpy()
    
    ra1=catalog1['ra'].to_numpy()
    dec1=catalog1['dec'].to_numpy()
    
    ra2=catalog2['ra'].to_numpy()
    dec2=catalog2['dec'].to_numpy()
    
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra1, dec=dec1, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra2, dec=dec2, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process_cross(kcat,gcat)
    
    return(np.array([kg.xi,kg.npairs]))

In [13]:
def get_ke_true(catalog,key):
    
    
    
    delta_g1=catalog['true_cm_g_1'].to_numpy()
    delta_g2=catalog['true_cm_g_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process_cross(kcat,gcat)
    xi=kg.xi*kg.npairs
    return(np.array([xi,kg.npairs]))

In [14]:
def get_ke_true_cross(catalog1,catalog2,key):
    
 

    k=catalog1[key].to_numpy()
    
    delta_g1=catalog2['true_cm_g_1'].to_numpy()
    delta_g2=catalog2['true_cm_g_2'].to_numpy()
    
    ra1=catalog1['ra'].to_numpy()
    dec1=catalog1['dec'].to_numpy()
    
    ra2=catalog2['ra'].to_numpy()
    dec2=catalog2['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra1, dec=dec1, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra2, dec=dec2, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process_cross(kcat,gcat)
    
    return(np.array([kg.xi,kg.npairs]))

In [15]:
tile_list=df_riz['meas_tilename'].unique()
        
def jk_resample(df):
    sample_dict={}
    for i,tile in tqdm(enumerate(tile_list)):
        jk_sample=df[(df['meas_tilename']==tile)]
        sample_dict[tile]=jk_sample
    return(sample_dict)

In [ ]:
ind_samples_for_keys={}
for key in keys:
    ind_samples_for_keys[key]=jk_resample(df_riz)

In [17]:
def get_corr_matrix(df,key):
    n = len(tile_list)
    corr_matrix= [ [ None for i in range(n) ] for j in range(n) ] 
    corr_matrix_true=[ [ None for i in range(n) ] for j in range(n) ] 
    for i,tile_i in tqdm(enumerate(tile_list)):
        for j,tile_j in enumerate(tile_list):
            if i==j:
                corr_matrix[i][j]=get_ke(ind_samples_for_keys[key][tile_i],key)
                corr_matrix_true[i][j]=get_ke_true(ind_samples_for_keys[key][tile_i],key)
#                 print(corr_matrix[i][j])
#                 print(corr_matrix_true[i])
            if i!=j:
                corr_matrix[i][j]=get_ke_cross(ind_samples_for_keys[key][tile_i],ind_samples_for_keys[key][tile_j],key)
                corr_matrix_true[i][j]=get_ke_true_cross(ind_samples_for_keys[key][tile_i],ind_samples_for_keys[key][tile_j],key)
#                 print(corr_matrix[i][j])
#                 print(corr_matrix_true[i][j])
    corr_matrix=np.array(corr_matrix)
    corr_matrix_true=np.array(corr_matrix_true)
    np.save("/home/cz136/project/balrog/data/matrices/corr_{}".format(key),corr_matrix)
    np.save("/home/cz136/project/balrog/data/matrices/corr_true_{}".format(key),corr_matrix_true)
    return(corr_matrix,corr_matrix_true)

In [ ]:
corr_matrices_dict_tuple={}
for key in keys:
    corr_matrices_dict_tuple[key]=get_corr_matrix(df_riz,key)

In [27]:
def get_jk_from_matrix(matrix_tuple):
    def get_sub_matrix(matrix,i,j):
        sub_matrix=[[entry for col_index,entry in enumerate(row) if col_index!=j] for row_index,row in enumerate(matrix) if row_index!=i]
        return (sub_matrix)
    def get_jked_sum(matrix,i):
        sub_matrix=np.array(get_sub_matrix(matrix,i,i))
        jk_sum=np.sum(np.sum(sub_matrix,axis=0),axis=0)
        return (jk_sum)
    corr_matrix, corr_matrix_true=matrix_tuple
    jk_xi=[None]*len(corr_matrix)
    jk_xi_true=[None]*len(corr_matrix)
    for i in tqdm(range(len(corr_matrix))):
        corr=get_jked_sum(corr_matrix,i)
        corr_true=get_jked_sum(corr_matrix_true,i)
        jk_xi[i]=corr[0]/corr[1]
        jk_xi_true[i]=corr_true[0]/corr_true[1]
    return(jk_xi,jk_xi_true)

In [ ]:
jk_list={}
for key in keys[1:4]:
    print(key)
    jk_list[key]=get_jk_from_matrix(corr_matrices_dict_tuple[key])

In [ ]:
def get_chisqr(covariance,y_value):
    return(np.dot(np.dot(y_value,np.linalg.inv(covariance)),y_value))

def get_cov(jk_samples_for_key):
    
    xi_array=np.array(jk_samples_for_key[0])
    xi_true_array=np.array(jk_samples_for_key[1])
    
    diff_array=xi_array-xi_true_array
    
    n = len(diff_array)
    
    emp_average=1/n*np.sum(diff_array,axis=0)
    nd = len(diff_array[0])
    cov=np.empty([nd,nd])
    for row in range(nd):
        for column in range(nd):
            cov[row][column]=\
            (n-1)/n * np.sum((diff_array[:,row]-emp_average[row])*(diff_array[:,column]-emp_average[column]))
    return (cov)

def get_y_value(key):
    y_value = get_ke(df_riz,key)[0]/ get_ke(df_riz,key)[1]
    return (y_value)

In [69]:
cov = get_cov(ebv)
y_value=get_y_value('ebv_planck13')

In [70]:
get_chisqr(cov,y_value)

3.780098790584814

In [32]:
star_density_cov=get_cov(jk_list['star_density_piffv23'])
y_value=get_y_value('star_density_piffv23')

In [33]:
get_chisqr(star_density_cov,y_value)

6.3195115328307